## Reinforcement Learning Models

### Imports and Utils

In [28]:
"""
Importing necessary libraries
"""
import pygame
import neat
import os
import numpy as np
import time

import pickle

In [29]:
"""
Defining Utilities for the Game
"""
scale = 0.5
WIN_WIDTH = 1920 * scale
WIN_HEIGHT = 1080 * scale
CAR_WIDTH = 40 * scale
CAR_HEIGHT = 40 * scale

CAR_IMG = pygame.transform.scale(pygame.image.load(os.path.join("Images", "car.png")), (CAR_WIDTH, CAR_HEIGHT))
ROAD_IMGS = []
for i in range(1, 6):
    ROAD_IMGS.append(pygame.transform.scale(pygame.image.load(os.path.join("Images", f"map{i}.png")), (int(WIN_WIDTH), int(WIN_HEIGHT))))
BORDER_COLOR = (255, 255, 255, 255)

In [30]:
"""
Defining Car Class
"""
class Car:
    """
    The Car Class
    """
    
    IMGS = CAR_IMG
    ROT_VEL = 10

    def __init__(self, position):
        """
        A Constructor for the Car Class

        position: list: The position of the car
        """

        self.position = position
        self.tilt = 0
        self.vel = 0

        self.image = self.IMGS
        self.center = (self.position[0] + self.image.get_width()//2, self.position[1] + self.image.get_height()//2)
        self.radars = []
        self.beams = []
        self.thetas = [90, 45, 0, -45, -90]

        self.dist = 0

    def engine(self, action):
        """
        The Engine of the Car

        action: list: The action to be taken by the car
        """

        if action[0] == 1:
            self.vel += 2
            self.vel = min(self.vel, 15)
        if action[1] == 1:
            self.vel -= 2
            self.vel += 0 if self.vel > 5 else 2
        if action[2] == 1:
            self.tilt += self.ROT_VEL
            self.tilt = self.tilt % 360
        if action[3] == 1:
            self.tilt -= self.ROT_VEL
            self.tilt = self.tilt % 360

    def move(self):
        """
        Update the position of the car
        """

        self.dist += self.vel
        self.position[0] += self.vel*np.cos(np.radians(self.tilt))
        self.position[1] -= self.vel*np.sin(np.radians(self.tilt))
        self.center = (self.position[0] + self.image.get_width()//2, self.position[1] + self.image.get_height()//2)
    
    def draw(self, window):
        """
        Draw the car on the window

        window: pygame.Surface: The window to draw the car on
        """

        rotated_image = pygame.transform.rotate(self.image, self.tilt)
        new_rect = rotated_image.get_rect(center=self.image.get_rect(topleft=self.position).center)
        window.blit(rotated_image, new_rect.topleft)

    def get_radars(self, map):
        """
        Get the radars of the car

        map: pygame.Surface: The map to get the radars from
        """
        
        self.radars = []
        self.beams = []
        for i in self.thetas:
            length = 0
            x = int(self.center[0] + length*np.cos(np.radians(self.tilt + i)))
            y = int(self.center[1] - length*np.sin(np.radians(self.tilt + i)))
            while x < WIN_WIDTH and y < WIN_HEIGHT and not map.get_at((x, y)) == BORDER_COLOR and length < 300:
                length += 1
                x = int(self.center[0] + length*np.cos(np.radians(self.tilt + i)))
                y = int(self.center[1] - length*np.sin(np.radians(self.tilt + i)))
            self.radars.append((x, y))
            self.beams.append(length)

    def get_mask(self):
        """
        Return: pygame.mask: The mask of the car
        """

        return pygame.mask.from_surface(self.image)
    


In [31]:
class Map:
    """
    The Map Class
    """
    
    IMGS = ROAD_IMGS

    def __init__(self, road, position = [0, 0]):
        """
        A Constructor for the Map Class

        road: int: The road to be used
        position: list: The position of the map
        """

        self.position = position
        self.image = self.IMGS[road]

        self.map_mask = self.get_mask()

    def collide(self, car):
        """
        Check if the car collides with the map

        car: Car: The car to check collision with
        """

        car_mask = car.get_mask()

        offset = (car.position[0] - self.position[0], car.position[1] - self.position[1])
        overlap = self.map_mask.overlap(car_mask, offset)

        return overlap is not None

    def get_mask(self):
        """
        Return: pygame.mask: The mask of the map
        """

        return pygame.mask.from_threshold(self.image, (255, 255, 255), (10, 10, 10, 255))     

In [46]:
"""
Game Structure and Functionality
"""
pygame.font.init()
RESTART_X = 830 * scale
RESTART_Y = 920 * scale
MAP_IND = 4

class RacingCarGame:
    """
    The Racing Car Game Class
    """

    def __init__(self, window, car, map):
        """
        A Constructor for the Racing Car Game Class

        window: pygame.Surface: The window to display the game on
        car: Car: The car to be used in the game
        map: Map: The map to be used in the game
        """

        self.window = window
        self.car = car
        self.map = map

        self.clock = pygame.time.Clock()
        self.running = True
        self.score = 0

    def gameplay(self):
        """
        The gameplay function
        """

        game_over = False
        while self.running:
            self.clock.tick(30)
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.running = False
                if event.type == pygame.KEYDOWN:
                    if game_over and event.key == pygame.K_SPACE:
                        self.car = Car([RESTART_X, RESTART_Y])
                        self.map = Map(MAP_IND, [0, 0])
                        self.score = 0
                        game_over = False
                    if event.key == pygame.K_w:
                        self.car.engine([1, 0, 0, 0])
                    if event.key == pygame.K_s:
                        self.car.engine([0, 1, 0, 0])
                    if event.key == pygame.K_a:
                        self.car.engine([0, 0, 1, 0])
                    if event.key == pygame.K_d:
                        self.car.engine([0, 0, 0, 1])

            if not game_over:
                self.car.move()
                self.score = self.car.dist // 100
                if self.map.collide(self.car):
                    game_over = True

            if game_over:
                self.Game_Over()
            else:
                self.display(DRAW_LINES = False)
        pygame.quit()

    def display(self, DRAW_LINES = False):
        """
        Display the game on the window

        DRAW_LINES: bool: Whether to draw the lines or not
        """

        self.window.blit(self.map.image, (0, 0))
        self.car.draw(self.window)

        if DRAW_LINES:
            self.car.get_radars(self.map.image)
            for radar in self.car.radars:
                pygame.draw.line(self.window, (255, 0, 0), self.car.center, radar, 2)
                pygame.draw.circle(self.window, (255, 0, 0), radar, 5)

        text = pygame.font.SysFont("norwester", 25).render(f"Score: {self.score}", 1, (0, 0, 0))
        self.window.blit(text, (10, 10))
        pygame.display.update()

    def Game_Over(self):
        """
        Display the Game Over Screen
        """
        self.window.blit(self.map.image, (0, 0))
        self.car.draw(self.window)
        text = pygame.font.SysFont("norwester", 25).render(f"Score: {self.score}", 1, (0, 0, 0))
        self.window.blit(text, (10, 10))
        text = pygame.font.SysFont("norwester", 50).render("Game Over", 1, (0, 0, 0))
        self.window.blit(text, (WIN_WIDTH//2 - text.get_width()//2, WIN_HEIGHT//2 - text.get_height()//2))
        pygame.display.update()

### Defining Game Structure

In [21]:
def main():
    car = Car([RESTART_X, RESTART_Y])
    map = Map(MAP_IND, [0, 0])
    window = pygame.display.set_mode((WIN_WIDTH, WIN_HEIGHT))

    pygame.font.init()
    game = RacingCarGame(window, car, map)
    game.gameplay()

main()

### Model Creation and Training

In [33]:
class CompleteGenome(Exception):
    """
    The Complete Genome Exception
    """

    def __init__(self, genome, net, score):
        """
        A Constructor for the Complete Genome Exception

        genome: neat.genome: The genome of the car
        net: neat.nn.FeedForwardNetwork: The neural network of the car
        score: int: The score of the car
        """
        
        self.genome = genome
        self.net = net
        self.score = score

In [34]:
"""
Game Structure and Functionality
"""
pygame.font.init()
RESTART_X = 830 * scale
RESTART_Y = 920 * scale

class RacingCarGameAI:
    """
    The Racing Car Game AI Class
    """

    TIMEOUT = 1

    def __init__(self, window, cars, map, genes, nets):
        """
        A Constructor for the Racing Car Game AI Class

        window: pygame.Surface: The window to display the game on
        cars: list: The cars to be used in the game
        map: Map: The map to be used in the game
        genes: list: The genes of the cars
        nets: list: The neural networks of the cars
        """

        self.window = window
        self.map = map

        self.cars = cars
        self.genes = genes
        self.nets = nets

        self.clock = pygame.time.Clock()
        self.running = True
        self.gen = 0
        self.scores = [0 for _ in range(len(self.cars))]
        self.is_alive = [True for _ in range(len(self.cars))]

    def gameplay(self, play = False):
        """
        The gameplay function

        play: bool: Whether to play the game or not
        """

        last_time = time.time()
        if_raise = False
        game_over = False
        while self.running:
            self.clock.tick(60)
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.running = False
                    best_ind = np.argmax(self.scores)
                    if_raise = not play
                    break
                if play and event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_SPACE:
                        self.cars = [Car([RESTART_X, RESTART_Y])]
                        self.map = Map(MAP_IND, [0, 0])
                        self.scores = [0]
                        self.is_alive = [True]
                        game_over = False

            current_time = time.time()
            if (not play) and (current_time - last_time > self.TIMEOUT):
                min_ind = np.argmin(self.scores)

                self.cars.pop(min_ind)
                self.genes.pop(min_ind)
                self.nets.pop(min_ind)
                self.scores.pop(min_ind)
                self.is_alive.pop(min_ind)

                last_time = current_time
                
            if not game_over:
                if not np.sum(self.is_alive):
                    self.running = False
                    break
                    
                for i, cars in enumerate(self.cars):
                    if self.is_alive[i]:
                        cars.get_radars(self.map.image)
                        output = self.nets[i].activate(cars.beams)
                        output = [1 if x > 0.5 else 0 for x in output]
                        cars.engine(output)
                        cars.move()

                        if self.map.collide(cars):
                            if not play:
                                self.genes[i].fitness -= 2
                                self.is_alive[i] = False
                                continue
                            else:
                                self.is_alive[i] = False
                                game_over = True

                        self.scores[i] = cars.dist // 100
                        self.genes[i].fitness = cars.dist + cars.vel

            if not play:
                self.display(DRAW_LINES = True)
            elif not game_over:
                self.display(DRAW_LINES = False)
            else:
                self.Game_Over(DRAW_LINES = False)

        pygame.quit()
        if if_raise:
            raise CompleteGenome(self.genes[best_ind], self.nets[best_ind], self.scores[best_ind])

    def display(self, DRAW_LINES = False):
        """
        Display the game on the window

        DRAW_LINES: bool: Whether to draw the lines or not
        """

        self.window.blit(self.map.image, (0, 0))
        for car in self.cars:
            car.draw(self.window)

        if DRAW_LINES and np.sum(self.is_alive) > 0:
            for i, car in enumerate(self.cars):
                if self.is_alive[i]:
                    car.get_radars(self.map.image)
                    for radar in car.radars:
                        pygame.draw.line(self.window, (255, 0, 0), car.center, radar, 2)
                        pygame.draw.circle(self.window, (255, 0, 0), radar, 5)

        text = pygame.font.SysFont("norwester", 25).render(f"Generation: {self.gen}", 1, (0, 0, 0))
        self.window.blit(text, (10, 10))
        text = pygame.font.SysFont("norwester", 25).render(f"Max Score: {np.max(self.scores)}", 1, (0, 0, 0))
        self.window.blit(text, (10, 40))
        text = pygame.font.SysFont("norwester", 25).render(f"Alive: {np.sum(self.is_alive)}", 1, (0, 0, 0))
        self.window.blit(text, (10, 70))
        
        pygame.display.update()
    
    def Game_Over(self, DRAW_LINES = False):
        """
        Display the Game Over Screen

        DRAW_LINES: bool: Whether to draw the lines or not
        """
        
        self.window.blit(self.map.image, (0, 0))
        for car in self.cars:
            car.draw(self.window)

        if DRAW_LINES and np.sum(self.is_alive) > 0:
            for i, car in enumerate(self.cars):
                if self.is_alive[i]:
                    car.get_radars(self.map.image)
                    for radar in car.radars:
                        pygame.draw.line(self.window, (255, 0, 0), car.center, radar, 2)
                        pygame.draw.circle(self.window, (255, 0, 0), radar, 5)

        text = pygame.font.SysFont("norwester", 25).render(f"Generation: {self.gen}", 1, (0, 0, 0))
        self.window.blit(text, (10, 10))
        text = pygame.font.SysFont("norwester", 25).render(f"Max Score: {np.max(self.scores)}", 1, (0, 0, 0))
        self.window.blit(text, (10, 40))
        text = pygame.font.SysFont("norwester", 25).render(f"Alive: {np.sum(self.is_alive)}", 1, (0, 0, 0))
        self.window.blit(text, (10, 70))

        text = pygame.font.SysFont("norwester", 50).render("Game Over", 1, (0, 0, 0))
        self.window.blit(text, (WIN_WIDTH//2 - text.get_width()//2, WIN_HEIGHT//2 - text.get_height()//2))
        
        pygame.display.update()

In [35]:
local_dir = os.getcwd()
config_path = os.path.join(local_dir, "Configs/config_RCAI.txt")
config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet, neat.DefaultStagnation, config_path)

population = neat.Population(config)
population.add_reporter(neat.StdOutReporter(True))
stats = neat.StatisticsReporter()
population.add_reporter(stats)
generation = 0

def main(genomes, config):
    """
    The main function to run the game

    genomes: list: The genomes of the cars
    config: neat.config.Config: The configuration of the game
    """
    
    global generation

    cars = []
    genes = []
    nets = []

    for _, g in genomes:
        net = neat.nn.FeedForwardNetwork.create(g, config)
        nets.append(net)
        cars.append(Car([RESTART_X, RESTART_Y]))
        g.fitness = 0
        genes.append(g)

    road = Map(MAP_IND, [0, 0])
    window = pygame.display.set_mode((WIN_WIDTH, WIN_HEIGHT))

    try:
        pygame.font.init()
        game = RacingCarGameAI(window, cars, road, genes, nets)
        game.gen = generation
        game.gameplay()
    except CompleteGenome as e:
        raise e

    generation += 1

try:
    winner = population.run(main, 100)
except CompleteGenome as e:
    winner = e.genome
print(f"The Best Genome:\n{winner}")


 ****** Running generation 0 ****** 

Population's average fitness: 68.83333 stdev: 111.67843
Best fitness: 324.00000 - size: (4, 20) - species 1 - id 23
Average adjusted fitness: 0.212
Mean genetic distance 0.959, standard deviation 0.199
Population of 30 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0    30    324.0    0.212     0
Total extinctions: 0
Generation time: 24.821 sec

 ****** Running generation 1 ****** 

Population's average fitness: 158.13333 stdev: 137.47454
Best fitness: 339.00000 - size: (4, 19) - species 1 - id 53
Average adjusted fitness: 0.466
Mean genetic distance 0.991, standard deviation 0.248
Population of 30 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1    30    339.0    0.466     0
Total extinctions: 0
Generation time: 13.353 sec (19.087 average)

 ****** Running generation 2 ****** 

Population's average fitness: 174

In [36]:
# Saving the winner genome
with open(f"Models/RacingCarAI_{MAP_IND}.pkl", "wb") as genome_file:
    pickle.dump(winner, genome_file)

### Genome Loading

In [44]:
with open(f"Models/RacingCarAI_{MAP_IND}.pkl", "rb") as genome_file:
    genome = pickle.load(genome_file)

local_dir = os.getcwd()
config_path = os.path.join(local_dir, "Configs/config_RCAI.txt")
config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet, neat.DefaultStagnation, config_path)

model = neat.nn.FeedForwardNetwork.create(genome, config)

In [45]:
def main():
    cars = [Car([RESTART_X, RESTART_Y])]
    genes = [genome]
    nets = [model]

    map = Map(MAP_IND, [0, 0])
    window = pygame.display.set_mode((WIN_WIDTH, WIN_HEIGHT))

    pygame.font.init()
    game = RacingCarGameAI(window, cars, map, genes, nets)
    game.gameplay(play = True)

main()